In [112]:
import requests
import pandas as pd
import urllib
from bs4 import BeautifulSoup
from sqlalchemy import create_engine

Define function: get_source

In [113]:
def get_source(url):
    """Return the source code for the provided URL. 

    Args: 
        url (string): URL of the page to scrape.

    Returns:
        response (object): HTTP response object from requests_html. 
    """

    try:
        r = requests.get(url)
        content = r.text
        return content

    except requests.exceptions.RequestException as e:
        print(e)

Set up variables: query, status, url_start, url_end

In [146]:
search_term = "data"
url = "https://www.digitalmarketplace.service.gov.uk/digital-outcomes-and-specialists/opportunities"


In [132]:
url_full = url + "?statusOpenClosed=open" + "&q=" + search_term_html + str("1") + "&page=" + page + "" 


https://www.digitalmarketplace.service.gov.uk/digital-outcomes-and-specialists/opportunities?page=1&statusOpenClosed=open&q=data


Loop through pages, call get_source, and extract opportunity title and hyperlink 

In [133]:
url_full = url_start + str(1) + url_end 
content = get_source(url_full)
soup = BeautifulSoup(content, "lxml")
#print(soup)
#print(soup.find("p", class_= "app-search-summary govuk-body-s").span.text)

In [142]:
def get_opps(url, search_term, open_closed):
    """Return the opps from the provided URL prefix and . 

    Args: 
        url (string): URL of the page to scrape.

    Returns:
        response (object): HTTP response object from requests_html. 
    """

opp_list = []
for page in range(1,1000):
    # create string
    search_term_html = urllib.parse.quote_plus(search_term)
    url_full = url + "?statusOpenClosed=open" + "&q=" + search_term_html + "&page=" + str(page) + "" 
    # retreive HTML content 
    content = get_source(url_full)
    # create BeautifulSoup soup object 
    soup = BeautifulSoup(content, "lxml")
    # if no page exists for the page loop, break
    if "Page could not be found" in soup.find("title").text:
        break
    # if no opportunities returned within URL HTML, break 
    elif soup.find("p", class_= "app-search-summary govuk-body-s").span.text == "0":
        print("no results")
        break
    # if opportunitues are returned and page exists on page loop, retrieve relevant information 
    else: 
        print(f"page {page} still in range")
        sections = soup.find_all("li", class_ = "app-search-result")
        #print(sections)
        for section in sections:
            opp_name = section.h2.a.text
            opp_url = section.h2.a["href"]
            opp_list.append({"opp_name" : opp_name, "opp_url" : opp_url})

return opp_list


page 1 still in range


In [143]:
print(opp_list)

[{'opp_name': 'DWP Identity Business Analyst 3', 'opp_url': '/digital-outcomes-and-specialists/opportunities/20081'}, {'opp_name': 'DWP Identity Content Designer', 'opp_url': '/digital-outcomes-and-specialists/opportunities/20039'}]


In [144]:
df = pd.DataFrame(opp_list)
df.reset_index(drop=True)

# Create a database engine
engine = create_engine('sqlite:///test.db')

# Drop the table if it already exists
engine.execute('DROP TABLE IF EXISTS my_table')

# Write the dataframe to a table in the database
df.to_sql('my_table', engine)

# Query the database using SQL syntax
query = 'SELECT opp_name, opp_url FROM my_table'
results = pd.read_sql(query, engine)

# # View the results as a Pandas dataframe
results

,opp_name,opp_url
0,DWP Identity Business Analyst 3,/digital-outcomes-and-specialists/opportunitie...
1,DWP Identity Content Designer,/digital-outcomes-and-specialists/opportunitie...


In [ ]:
def get_opps():
    """Return all opps from the provided URL 

    Args: 
        url (string): URL of the page to scrape.

    Returns:
        list of dictionaries: [{"opp_name":"opp name string", "opp_url": "opp url string"}, {n:n, n:n}]
    """

opp_list = []
for page in range(1,1000):
    # create string
    url_full = url_start + str(page) + url_end
    # retreive HTML content 
    content = get_source(url_full)
    # create BeautifulSoup soup object 
    soup = BeautifulSoup(content, "lxml")
    # if no page exists for the page loop, break
    if "Page could not be found" in soup.find("title").text:
        break
    # if no opportunities returned within URL HTML, break 
    elif soup.find("p", class_= "app-search-summary govuk-body-s").span.text == "0":
        print("no results")
        break
    # if opportunitues are returned and page exists on page loop, retrieve relevant information 
    else: 
        print(f"page {page} still in range")
        sections = soup.find_all("li", class_ = "app-search-result")
        #print(sections)
        for section in sections:
            opp_name = section.h2.a.text
            opp_url = section.h2.a["href"]
            opp_list.append({"opp_name" : opp_name, "opp_url" : opp_url})

return opp_list
